In [1]:
"""
Script to extract recent game review from IGN
"""

'\nScript to extract recent game review from IGN\n'

In [2]:
import os
os.getcwd()
os.chdir("/Users/natepruitt/Projects/robocritic/")

In [3]:
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
# Imports
import requests
from settings import settings
from bs4 import BeautifulSoup
from helper_functions.db_connection import connection
from helper_functions.query_functions import get_publisher_by_id, get_game_by_title, get_review_by_game_id_and_publisher_id, GameReleaseDate, Reviews
from helper_functions.igdb_service import IGDBService
from helper_functions.scraping_utils import get_latest_reviews_from_ign, get_game_review_content_from_ign, get_game_release_date_from_metacritic
from helper_functions.write_functions import create_game, create_review, write_review_pros_to_db, write_review_cons_to_db
from helper_functions.openai_service import OpenAIService
from datetime import datetime

The error '2003 (HY000): Can't connect to MySQL server on 'localhost:3306' (61)' occurred


In [5]:
# retrieve the game release date from igdb API
igdb_service = IGDBService(connection)

# Instantiate OpenAI service
openai_service = OpenAIService()

In [6]:
# Pull latest games from IGN
latest_reviews = get_latest_reviews_from_ign()

In [10]:
game_title = "Prince of Persia: The Lost Crown"
review_publisher = "IGN"
review = Reviews().get_review_by_game_title_and_reviewer_name("Supernormal", review_publisher)
review

{'Pros': {'L': [{'S': 'Beautifully done scares, especially when played alone in a dark room with headphones'},
   {'S': 'Tense atmosphere in the exploration of the creepy house'},
   {'S': 'Randomized events and spooky interactions add to the unsettling experience'},
   {'S': 'Voice recognition system adds an intriguing feature'}]},
 'RoboScore': {'N': '7'},
 'ReviewPublisherName': {'S': 'IGN'},
 'Cons': {'L': [{'S': 'Underwhelming first half with strange choices and dull opening cinematic'},
   {'S': 'Awful dialogue and unconvincing delivery by the character'},
   {'S': 'Clumsy and predictable story with lack of replayability'},
   {'S': 'Voice recognition system not functioning properly'}]},
 'GameTitle': {'S': 'Supernormal'},
 'GameReleaseDate': {'S': '2024-01-08'}}

In [8]:
latest_reviews[1:3]

[{'title': 'Prince of Persia: The Lost Crown',
  'href': '/articles/prince-of-persia-the-lost-crown-review'},
 {'title': 'Supernormal', 'href': '/articles/supernormal-review'}]

In [9]:
# For each review, create a game entry if does not exist
for review in latest_reviews[1:3]:
    game_title = review['title']
    print(game_title)
    game_release_date = None
    try:
        game_release_date = igdb_service.get_first_release_date_by_title(game_title).strftime("%Y-%m-%d")
    except requests.exceptions.HTTPError as e:
        print(e)
    except ValueError as e:
        print(e)
    # if not game_release_date:
    #     # if pulling release_date from igdb service failed, try metacritic
    try:
        game_release_date = get_game_release_date_from_metacritic(game_title)
        game_release_date = game_release_date.strftime("%Y-%m-%d")
    except requests.exceptions.HTTPError as e:
        print(e)
    except ValueError as e:
        print(e)

    if not game_release_date:
        # If fail to pull release date from meta critic and API, mark as no release date
        game_release_date = 'NoRelease'
    
    print(game_release_date)
    print(game_release_date + '_' + game_title)

    games_response = GameReleaseDate().write_item(game_release_date + '_' + game_title)
    print(games_response)
    print(f"Writing review to db for {review['title']}")
    # Scrape review from site. 
    # Would be nice if review already exists NOT to call openAI api. 
    # Check if review exists for review. 
    review_in_db = Reviews().get_review_by_game_title_and_reviewer_name(game_title=game_title, review_publisher_name="IGN")
    if review_in_db:
        # if review does not exist, continue to next in loop.
        print(f"Review found for game {game_title} published by IGN")
        continue

    review_url = "https://www.ign.com" + review['href']
    print(f"Scrape review from {review_url}")
    game_review_content = get_game_review_content_from_ign(review_url)

    
    # Get roboscore from open ai
    score_json = openai_service.assign_score_to_content(game_review_content)
    score = score_json["score"]
        
    # Send content to OpenAI API. 
    pros_cons_json = openai_service.extract_review_pros_and_cons(game_review_content)

    # Parse pros cons list
    # Create a list of tuples with review_id, text
    pros = pros_cons_json["pros"]
    cons = pros_cons_json["cons"]

    reviews_response = Reviews().write_item(game_title=game_title,
                         review_publisher_name="IGN",
                         game_release_date=game_release_date,
                         list_of_pros=pros,
                         list_of_cons=cons,
                         roboscore=score)
    print(reviews_response)
    # Create review in DB
    # db_review = {"game_id": game_id, "url": review_url, "publisher_id": settings.ign_publisher_id, "robo_score": score}
    # print(f"Create review in db out of review object {db_review}")
    # review_id = create_review(connection, db_review)



Prince of Persia: The Lost Crown
2024-01-18
2024-01-18_Prince of Persia: The Lost Crown
Item already exists 2024-01-18_Prince of Persia: The Lost Crown
{'Error': {'Message': 'The conditional request failed', 'Code': 'ConditionalCheckFailedException'}, 'ResponseMetadata': {'RequestId': '35027E5FSN2MDPCN7URETFI4LBVV4KQNSO5AEMVJF66Q9ASUAAJG', 'HTTPStatusCode': 400, 'HTTPHeaders': {'server': 'Server', 'date': 'Thu, 18 Jan 2024 13:29:28 GMT', 'content-type': 'application/x-amz-json-1.0', 'content-length': '120', 'connection': 'keep-alive', 'x-amzn-requestid': '35027E5FSN2MDPCN7URETFI4LBVV4KQNSO5AEMVJF66Q9ASUAAJG', 'x-amz-crc32': '396270901'}, 'RetryAttempts': 0}, 'message': 'The conditional request failed'}
Writing review to db for Prince of Persia: The Lost Crown
Review found for game Prince of Persia: The Lost Crown published by IGN
Supernormal
2024-01-08
2024-01-08_Supernormal
Item already exists 2024-01-08_Supernormal
{'Error': {'Message': 'The conditional request failed', 'Code': 'Cond

In [ ]:
game = get_game_by_title(connection, 'The Finals')
print(game)

In [ ]:
the_finals_release_date = igdb_service.get_game_release_date_by_title('The Finals')

In [ ]:
create_game(connection, {"title": "The Finals", "release_date": the_finals_release_date})

In [ ]:
get_latest_reviews_from_ign()

In [ ]:
igdb_service.get_game_release_date_by_title("Resident Evil 4 Remake VR")

In [ ]:
game = []
game[0]

In [ ]:
r = requests.get('http://www.google.com/nothere')
r.status_code

In [ ]:
ign_latest_reviews_url = "https://www.ign.com/reviews/games"
header_mapping = {"User-Agent": "Mac Firefox"}

ign_latest_reviews_html = requests.get(
        ign_latest_reviews_url, headers=header_mapping
    )
ign_latest_reivews_soup = BeautifulSoup(
        ign_latest_reviews_html.content, "html.parser"
    )
latest_reviews_a_tag_list = ign_latest_reivews_soup.find(
        class_="main-content"
    ).find_all("a")

filter_latest_reviews_a_tag = filter(
        lambda x: bool(x.get("aria-label")), latest_reviews_a_tag_list
    )

filtered_a_tags = list(filter_latest_reviews_a_tag)


In [ ]:
filtered_a_tags[0].find(class_="item-publish-date").text

In [ ]:
from datetime import datetime

In [ ]:
datetime.strptime("5d ago", "%b %d, %Y").date().strftime("%Y-%m-%d")

In [ ]:
# If error thrown trying to convert datetime, parse as '5d ago'
"11d ago".split("d")

In [ ]:
datetime.

In [ ]:
GameReleaseDate().get_recent_games()